In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
df_ = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2009-2010")
df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom


In [3]:
def create_cltv_c(dataframe, profit=0.10):

    # Data Preparation
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    dataframe = dataframe[(dataframe['Quantity'] > 0)]
    dataframe.dropna(inplace=True)
    dataframe["TotalPrice"] = dataframe["Quantity"] * dataframe["Price"]
    cltv_c = dataframe.groupby('Customer ID').agg({'Invoice': lambda x: x.nunique(),
                                                   'Quantity': lambda x: x.sum(),
                                                   'TotalPrice': lambda x: x.sum()})
    cltv_c.columns = ['total_transaction', 'total_unit', 'total_price']
    
    # avg_order_value
    cltv_c['avg_order_value'] = cltv_c['total_price'] / cltv_c['total_transaction']
    
    # purchase_frequency
    cltv_c["purchase_frequency"] = cltv_c['total_transaction'] / cltv_c.shape[0]
    
    # repeat rate & churn rate
    repeat_rate = cltv_c[cltv_c.total_transaction > 1].shape[0] / cltv_c.shape[0]
    churn_rate = 1 - repeat_rate
    
    # profit_margin
    cltv_c['profit_margin'] = cltv_c['total_price'] * profit
    
    # Customer Value
    cltv_c['customer_value'] = (cltv_c['avg_order_value'] * cltv_c["purchase_frequency"])
    
    # Customer Lifetime Value
    cltv_c['cltv'] = (cltv_c['customer_value'] / churn_rate) * cltv_c['profit_margin']
    
    # Segment
    cltv_c["segment"] = pd.qcut(cltv_c["cltv"], 4, labels=["D", "C", "B", "A"])
    
    cltv_c.index = cltv_c.index.astype(int)

    return cltv_c

In [4]:
clv = create_cltv_c(df)
clv.head(20)

,total_transaction,total_unit,total_price,avg_order_value,purchase_frequency,profit_margin,customer_value,cltv,segment
Customer ID,,,,,,,,,
12346,11,70,372.86000,33.89636,0.00255,37.28600,0.08643,9.78357,C
12347,2,828,1323.32000,661.66000,0.00046,132.33200,0.30675,123.23546,B
12348,1,373,222.16000,222.16000,0.00023,22.21600,0.05150,3.47326,D
12349,3,993,2671.14000,890.38000,0.00070,267.11400,0.61918,502.11041,A
12351,1,261,300.93000,300.93000,0.00023,30.09300,0.06976,6.37290,D
12352,2,188,343.80000,171.90000,0.00046,34.38000,0.07969,8.31798,C
12353,1,192,317.76000,317.76000,0.00023,31.77600,0.07366,7.10566,C
12355,1,303,488.21000,488.21000,0.00023,48.82100,0.11317,16.77333,C
12356,3,1826,3562.25000,1187.41667,0.00070,356.22500,0.82574,893.00669,A


In [5]:
clv.groupby("segment").agg({"count", "sum", "mean"})

total_transaction                 total_unit                      \
                    count    sum     mean      count      sum       mean   
segment                                                                    
D                    1079   1326  1.22892       1079   117616  109.00463   
C                    1078   2160  2.00371       1078   305135  283.05659   
B                    1078   4063  3.76902       1078   733211  680.15863   
A                    1079  11666 10.81186       1079  4383262 4062.33735   

        total_price                          avg_order_value               \
              count           sum       mean           count          sum   
segment                                                                     
D              1079  192265.13000  178.18826            1079 169805.46550   
C              1078  513016.45300  475.89652            1078 317422.35514   
B              1078 1219605.20000 1131.35918            1078 420127.12365   
A              1079 6907116.49100 6401.40546            1079 723972.22347   

                  purchase_frequency                 profit_margin  \
             mean              count     sum    mean         count   
segment                                                              
D       157.37300               1079 0.30737 0.00028          1079   
C       294.45487               1078 0.50070 0.00046          1078   
B       389.72832               1078 0.94182 0.00087          1078   
A       670.96592               1079 2.70422 0.00251          1079   

                               customer_value                     cltv  \
                 sum      mean          count        sum    mean count   
segment                                                                  
D        19226.51300  17.81883           1079   44.56772 0.04130  1079   
C        51301.64530  47.58965           1078  118.91897 0.11031  1078   
B       121960.52000 113.13592           1078  282.70867 0.26225  1078   
A       690711.64910 640.14055           1079 1601.09330 1.48387  1079   

                                    
                   sum        mean  
segment                             
D           2849.32790     2.64071  
C          18184.26342    16.86852  
B         103549.86329    96.05739  
A       25257295.04118 23408.05843